# CIS 545 Homework 3: Spark SQL

#### **Worth 100 points, due Tuesday November 1st, 10:00 PM EST**

Welcome to CIS 545 Homework 3! In this homework you will master Spark SQL. By the end, you'll be a star (not that you aren't already one). Over the next few days you will be using an EMR cluster to use Spark to manipulate the datasets about Amazon products and their reviews. 

## The Necessary Notes and Nags
Before we begin here are some important notes to keep in mind,


1.   **IMPORTANT!** I said it twice, it's really important. In this homework, we will be using AWS resources. You are given a quota ($100) to use for the entirety of the homework. There is a small chance you will use all this money, however it is important that at the end of every session, you **shut down your EMR cluster**.
2.   **Be sure you use Google Colab for this Homework** since we must connect to the EMR cluster and local Jupyter will have issues doing that. Using a Google Colab Notebook with an EMR cluster has two important abnormalities:
    * The first line of any cell in which you will use the spark session must be `%%spark`. Notice that all cells below have this.
    * You will, unfortunately, not be able to stop a cell while it is running. If you wish to do so, you will need to restart your cluster. See the Setup EMR Document for reference.
3.   You are **required** to use Spark SQL queries to handle the data in the assignment. Mastering SQL is more beneficial than being able to use Spark commands (functions) as it will show up in more areas of programming and data science/analytics than just Spark. Use the following [function list](https://spark.apache.org/docs/latest/api/sql/index.html#) to see all the SQL functions avaliable in Spark.
4.   Throughout the homework you will be manipulating Spark dataframes (sdfs). We do not specify any ordering on the final output. You are welcome to order your final tables in whatever way you deem fit. We will conduct our own ordering when we grade.
5. Based on the challenges you've faced in the previous homework, we are including information on the expected schema of your results.  Apache Spark is very fiddly but we hope this will help.
6. There are portions of this homework that are _very_ hard. We urge you start early to come to office hours and get help if you get stuck. But don't worry, I can see the future, and you've all got this.

With that said, let's dive in.




## Step 0: Set up EMR

Follow the [AWS Academy Getting Started](https://docs.google.com/document/d/1JPitLGaorjTbXjGsaoIHcLTu2cj8rjm5UNr9bSpZ72k/edit?usp=sharing) instructions.

Move on to Step 0.1 after you have completed all the steps in the document.

![ACME GIANT RUBBER BAND](https://pbs.twimg.com/media/DRqbJh7UMAE2z4o?format=jpg&name=4096x4096)


### Step 0.1: The Superfluous Setup

Run the following two cells. These will allow your colab notebook to connect to an use your EMR.

In [ ]:
#%%capture
!apt install libkrb5-dev
!pip install sparkmagic
!pip install -i https://test.pypi.org/simple/ penn-grader==0.5.0

In [ ]:
%load_ext sparkmagic.magics

### Step 0.2: The Sharp Spark

Now, connect your notebook to the EMR cluster you created. In the first cell, copy the link to the Master Public DNS specified in the setup document. You will need to add `http://` to the beginning of the address and the auth details to the end.

For example, if my DNS (directly from the AWS EMR console) is `ec2-3-15-237-211.us-east-2.compute.amazonaws.com` my address would be,

`http://ec2-3-15-237-211.us-east-2.compute.amazonaws.com -a cis545-livy -p password1 -t Basic_Access`

Insert this in the `# TODO # below`. For our example, the cell would read,

```
%spark add -s spark_session -l python -u http://ec2-3-15-237-211.us-east-2.compute.amazonaws.com -a cis545-livy -p password1 -t Basic_Access
```

In [ ]:
# TODO: Copy the line above, enter your Master Public DNS with the proper formatting and host, and update the password



Starting Spark application


ID,YARN Application ID,Kind,State,Spark UI,Driver log,User,Current session?
1,application_1665349480642_0002,pyspark,idle,Link,Link,None,✔


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

SparkSession available as 'spark'.


In [ ]:
# If you ever need to restart, you may need to...
# %spark delete -s my_session
#OR just factory reset runtime under the runtime tab
# %spark delete -s spark_session

### Step 0.3: Cluster Log

In order to keep track of clusters you have created and terminated as well as give us information about time spent on this assignment, please enter each date and time you created a cluster and the date and time you terminated the cluster. This will not impact your score.

EX: 

10/12 9:00am - 10/12 12:00pm

10/13 7:00pm - 10/13 9:00pm

...
    

TODO: Create cluster log

Enter your 8-digit Penn Key as an integer in the cell 
below. This will be used in the autograder.  **Please also update the cell below, with the same ID!**

In [ ]:
from penngrader.grader import *
STUDENT_ID = 99999999

In [ ]:
%%writefile config.yaml

grader_api_url: 'https://wyv616tp17.execute-api.us-east-1.amazonaws.com/default/Grader'
grader_api_key: 'Kd32fl3g3p917iM0zwjiO23Bitj4PO9ga4LektOa'

Writing config.yaml


In [ ]:
grader = PennGrader('config.yaml', 'CIS_5450_Fall22_HW3', STUDENT_ID, STUDENT_ID)

PennGrader initialized with Student ID: 99999999

Make sure this correct or we will not be able to store your grade


Run the above cells to setup the autograder. Make sure to have set your 8 digit Penn ID in the cell above. It will also import all the modules you need for the homework.


## Step 1: Data Wrangling, Cleaning, and Shaping [18 pts]

In this homework we will be working with data from Amazon regarding their products and customer reviews. There are three JSON datasets with information about the products and three JSON datasets with information about the customer reviews for these products. The three categories of products are: Gourmet & Grocery Food, Toys & Games, and Home & Kitchen. 


The data you will use is stored in an S3 bucket, a cloud storage service. You now need to download it onto the nodes of your [EMR cluster](https://docs.aws.amazon.com/emr/latest/ManagementGuide/emr-what-is-emr.html). 


*Citation for data:* 

- Image-based recommendations on styles and substitutes.
J. McAuley, C. Targett, J. Shi, A. van den Hengel
SIGIR, 2015

- Ups and downs: Modeling the visual evolution of fashion trends with one-class collaborative filtering.
R. He, J. McAuley
WWW, 2016

### Step 1.1: The Stupendous Schema

When loading data, Spark will try to infer its structure on its own. This process is faulty because Spark will sometimes infer the type incorrectly. Spark's ability to determine types is not reliable, thus you will need to define a schema for [] and .

A schema is a description of the structure of data. We have given you an example for the `products` datasets and you will be defining an explicit schema for the `reviews` datasets called ```schema_reviews```. 


In Spark, schemas are defined using a `StructType` object. This is a collection of data types, termed `StructField`'s, that specify the structure and variable type of each component of the dataset. For example, suppose we have the following simple JSON object,


```
{
 "student_name": "Data Wrangler",
 "GPA": 1.4,
 "courses": [
    {"department": "Computer and Information Science",
     "course_id": "CIS 545",
     "semester": "Fall 2021"},
    {"department": "Computer and Information Science",
     "course_id": "CIS 555",
     "semester": "Fall 2021"}
 ],
 "grad_year": 2022
 }
```

We would define its schema as follows,

```       
schema = StructType([
           StructField("student_name", StringType(), nullable=True),
           StructField("GPA", FloatType(), nullable=True),
           StructField("courses", ArrayType(
                StructType([
                  StructField("department", StringType(), nullable=True),
                  StructField("course_id", StringType(), nullable=True),
                  StructField("semester", StringType(), nullable=True)
                ])
           ), nullable=True),
           StructField("grad_year", IntegerType(), nullable=True)
         ])
```


Each `StructField` has the following structure: `(name, type, nullable)`. The `nullable` flag defines that the specified field may be empty. Your first task is to define the `schema` of the five `reviews` .json files. You can take a look at the schema for one of these files for reference. A smaller version of the `food_reviews` JSON dataset can be found here: https://drive.google.com/file/d/1mWYmGIhanT3fwB3cNkMSp67A6kkI_Mu3/view?usp=sharing




There is also no grading cell for this step.  But your JSON file won't load if it's wrong, so you have a way of testing.



In [ ]:
%%spark

from pyspark.sql.types import *

schema_products = StructType([
          StructField("asin", StringType(), nullable=True),
           StructField("title", StringType(), nullable=True),
           StructField("price", FloatType(), nullable=True),
           StructField("imUrl", StringType(), nullable=True),
           StructField("related", ArrayType(
                StructType([
                  StructField("also_bought", ArrayType(StringType()), nullable=True),
                  StructField("also_viewed", ArrayType(StringType()), nullable=True),
                  StructField("bought_together", ArrayType(StringType()), nullable=True)
                ])
           ), nullable=True),
           StructField("salesRank", MapType(StringType(), StringType()), nullable=True),
           StructField("brand", StringType(), nullable=True),
           StructField("categories", ArrayType(StringType()), nullable=True),
])

#TODO: Create a schema for the reviews datasets
schema_reviews = 



          
          

### Step 1.2: The Langorous Load [9 pts]

In the following cell, load the three `reviews` datasets and the three `products` datsets from your S3 bucket into Spark dataframes (sdf). If you have constructed `schema_reviews` and `schema_products` correctly, `spark.read.json()` will read in the datasets. The names of the datasets are listed below:

Reviews:

- `food_reviews.json`
- `home_kitchen_reviews.json`
- `toys_games_reviews.json`

Products:

- `food_metadata.json`
- `home_kitchen_meta.json`
- `toys_games_meta.json`



***You do not need to edit this cell***. If this doesn't work, go back to the prior cell and update your schema. Note that the cell below will load data even if your schema is incomplete and has left out some columns of the data, so be sure to check that you have included all of the fields from the JSON.

In [ ]:
%%spark

food_reviews_sdf = spark.read.json("s3://penn-cis545-files/food_reviews.json", schema=schema_reviews, multiLine = False, primitivesAsString = True)
home_kitchen_reviews_sdf = spark.read.json("s3://penn-cis545-files/home_kitchen_reviews.json", schema=schema_reviews, multiLine = False, primitivesAsString = True)
toys_games_reviews_sdf = spark.read.json("s3://penn-cis545-files/toys_games_reviews.json", schema=schema_reviews, multiLine = False, primitivesAsString = True)


In [ ]:
%%spark
# Let's print out the first few rows to see how the data looks like in tabular form
home_kitchen_reviews_sdf.show(5)

In [ ]:
%%spark

#TODO: Create a spark dataframe for food_prod_sdf, home_kitchen_prod_sdf, and toys_games_prod_sdf


The cell below shows how to run SQL commands on Spark tables. Use this as a template for all your SQL queries in this notebook. 

***You do not need to edit this cell***.

In [ ]:
import pandas as pd

In [ ]:
%%spark

# Create SQL-accesible table
food_reviews_sdf.createOrReplaceTempView("food_reviews")

# Declare SQL query to be excecuted
query = '''SELECT * 
           FROM food_reviews ORDER BY `asin` DESC LIMIT 10'''

# Save the output sdf of spark.sql() as answer_review_sdf and convert to Pandas
answer_review_sdf = spark.sql(query)
answer_review_sdf.show()


We can then copy the `answer_review_sdf` to Colab to submit to PennGrader...

In [ ]:
%spark -o answer_review_sdf

In [ ]:
answer_review_sdf

Now, you will implement the exact same thing yourself for `food_prod_sdf` and submit it to the grader. Select all columns from `food_prod` and order by the `asin` column (descending). Limit your output to only 10 rows and save your output dataframe to Colab as answer_prod_sdf. 

In [ ]:
%%spark

# Create SQL-accesible table


# Declare SQL query to be excecuted


# Save the output sdf of spark.sql() as answer_prod_sdf 



FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [ ]:
%spark -o answer_prod_sdf

In [ ]:
## AUTOGRADER 
#4 points
grader.grade(test_case_id = 'test_answer_prod', answer = answer_prod_sdf)

In the next cell, we want to find the ten most expensive food products. Create `food_prices_sdf` to fetch the data from `food_prod_sdf` table, returning rows with schema `(asin,title,price)` and ordered by price (descending). Limit your sdf to 10 rows. Save your final answer to Colab to submit to PennGrader.


In [ ]:
%%spark 

# TODO: create food_prices_sdf


In [ ]:
%spark -o food_prices_sdf

In [ ]:

# 5 points
grader.grade(test_case_id = 'test_food_price', answer = food_prices_sdf)

### Step 1.3: Further Cleaning [9 pts]

Before we move on to the later parts of this homework, we want to do some cleaning of our datasets. Currently, our datasets are divided based on their category (Food is in its own table, Home and Kitchen is its own table, etc.), but we eventually want to combine our dataframes so that they are easier to work with. If we do this, we want to be able to discern what category each product belongs to, but there currently does not exist any column that tells us the category of a product.


For `food_prod_sdf`, create a new column called `category` where every value in this column will be 'Food'. Name this dataframe as `food_prod_final_sdf`. Repeat this for `toys_games_prod_sdf` and `home_kitchen_prod_sdf.` In `toys_games_prod_sdf`, every value in the `category` column will be 'Toys & Games'. In `home_kitchen_prod_sdf`, every value in this column will be 'Home & Kitchen'. Name these dataframes as `toys_games_prod_final_sdf` and `home_kitchen_prod_final_sdf`


In [ ]:
%%spark

#TODO: Create food_prod_final_sdf




In [ ]:
%%spark
# Do not edit
answer_food_final_sdf = food_prod_final_sdf.alias('answer_food_final_sdf')

In [ ]:
%spark -o answer_food_final_sdf

In [ ]:
# 2 points
grader.grade(test_case_id = 'test_food_prod_final', answer = answer_food_final_sdf)

In [ ]:
%%spark

#TODO: Create home_kitchen_prod_final_sdf



In [ ]:
%%spark
# Do not edit
answer_home_kitchen_sdf = home_kitchen_prod_final_sdf.alias('answer_home_kitchen_sdf')

In [ ]:
%spark -o answer_home_kitchen_sdf

In [ ]:
# 2 points
grader.grade(test_case_id = 'test_home_kitchen_final', answer = answer_home_kitchen_sdf)

In [ ]:
%%spark

#TODO: Create toys_games_prod_final_sdf


In [ ]:
%%spark
# Do not edit
answer_toys_games_sdf = toys_games_prod_final_sdf.alias('answer_home_kitchen_sdf')

In [ ]:
%spark -o answer_toys_games_sdf

In [ ]:
# 2 points
grader.grade(test_case_id = 'test_toys_games_final', answer = answer_toys_games_sdf)


This part is given to you. Combine the newly created `food_prod_sdf`, `toys_games_prod_sdf`, and `home_kitchen_prod_sdf` into a single dataframe called `products_sdf`. Combine `food_reviews_sdf`, `toys_games_reviews_sdf`, and `home_kitchen_reviews_sdf` into a single dataframe called `reviews_sdf`.

***You do not need to edit the two cells below***.

In [ ]:
%%spark

# Create SQL-accesible table
food_prod_final_sdf.createOrReplaceTempView("food_prod_final")
home_kitchen_prod_final_sdf.createOrReplaceTempView("home_kitchen_prod_final")
toys_games_prod_final_sdf.createOrReplaceTempView("toys_games_prod_final")

# Declare SQL query to be excecuted
query = '''(SELECT * FROM food_prod_final)
           
           UNION All
           
           (SELECT * FROM home_kitchen_prod_final)
           
           UNION All
           
           (SELECT * FROM toys_games_prod_final)'''

# Save the output sdf of spark.sql() as products_sdf
products_sdf = spark.sql(query)

In [ ]:
%%spark

# Create SQL-accesible table
food_reviews_sdf.createOrReplaceTempView("food_reviews_final")
home_kitchen_reviews_sdf.createOrReplaceTempView("home_kitchen_reviews_final")
toys_games_reviews_sdf.createOrReplaceTempView("toys_games_reviews_final")

# Declare SQL query to be excecuted
query = '''(SELECT * FROM food_reviews_final)
           
           UNION All
           
           (SELECT * FROM home_kitchen_reviews_final)
           
           UNION All
           
           (SELECT * FROM toys_games_reviews_final)'''

# Save the output sdf of spark.sql() as reviews_sdf
reviews_sdf = spark.sql(query)



Join `reviews_sdf` and `products_sdf` on `asin` and name the resulting dataframe as `product_reviews_sdf`. Keep only the following columns from reviews_sdf: `reviewerID`, `asin`, `helpful`, `reviewText`, `overall`, `summary`, `reviewTime`, `unixReviewTime`. Keep the following columns from products_sdf: `title`, `category`, `price`.

In [ ]:
%%spark

#TODO: Create product_reviews_sdf


In [ ]:
%%spark

# Do not edit - run this after creating product_reviews_sdf
product_reviews_sdf.createOrReplaceTempView("product_reviews")
ans_product_reviews_sdf = spark.sql("SELECT * FROM product_reviews ORDER BY asin ASC LIMIT 10")

In [ ]:
%spark -o ans_product_reviews_sdf

In [ ]:


# 3 points
grader.grade(test_case_id = 'test_product_review', answer = ans_product_reviews_sdf)

## Step 2: Analysis! [26 pts]

### Step 2.1: Overall Opinion [5 pts]

We would like to find the average rating for products based on category. Using `product_reviews_sdf`, create `avg_rating_sdf` with schema `(category,avg_rating)`. Order your resulting dataframe by `avg_rating` (descending). Note that the column `overall` holds ratings for each product.


In [ ]:
%%spark

#TODO: Create avg_rating_sdf


In [ ]:
%spark -o avg_rating_sdf

In [ ]:


# 5 points
grader.grade(test_case_id = 'test_avg_rating', answer = avg_rating_sdf)

### Step 2.2: The Category Changes [10 pts]

We would like to find the percentage change from 2010 to 2011 in both average overall rating and average price for products based on category.

*Hint: Use SUBSTR on the reviewTime field to retrieve the correct year.*

Your task includes:


*   Finding the average overall rating for each category for the years 2010, 2011
*   Calculate the percentage change in rating for 2010 to 2011
*   Finding the average price for each category for the years 2010, 2011
*   Calculate the percentage change in price for 2010 to 2011
*   Return an sdf with the category, change_rating, change_price sorted in descending alphabetical order of category name.

Hint: Using multiple queries with intermediate dataframes may be helpful
```
+-------------------+--------------------+-------------+
|           category|       change_rating|.change_price|
+-------------------+--------------------|.------------|            
+-------------------+--------------------+.------------|
```

In [ ]:
%%spark
#TODO: Create pct_change_sdf


In [ ]:
%spark -o pct_change_sdf

In [ ]:

# 10 points
grader.grade(test_case_id = 'test_pct_change', answer = pct_change_sdf)

### Step 2.3: The Popular Products [11 pts]

When you navigate to the Amazon webpage, you are often recommended some bestsellers. These bestsellers are recommended to you based on several factors, one of them being positive ratings and reviews. Most popular products have tons of reviews and ratings from customers. We're going to look into popular products bought in a few categories to dive deeper into the consumer market culture. 

#### Step 2.3.1: The Terrific Toys [6 pts]

From the product_reviews_sdf, find the 5 most commonly reviewed products around Christmas in the Toys and Games category. In this case, a product reviewed in the month of December (12), will count as 'around Christmas'.

The resultant dataframe should contain the 5 most popular products and the number of reviews the product sorted in order of most to least popular in this format:


```
+-------------------+--------------------+
|               asin|         num_reviews|
+-------------------+--------------------|
+-------------------+--------------------+
```



In [ ]:
%%spark

#TODO: Create dec_toys_sdf


In [ ]:
%spark -o dec_toys_sdf

In [ ]:


# 6 points
grader.grade(test_case_id = 'test_dec_toys', answer = dec_toys_sdf)

#### Step 2.3.2: Healthy Home [5 pts]

We are interested in finding the products that are most frequently reviewed within the Home and Kitchen category. Find the top three months in which products from the Home and Kitchen category are most frequently reviewed during the years 2010 to 2014 (inclusive). As before, the `SUBSTR` function in SQL will be helpful for you in your query.

Return a dataframe with the schema `(month, num_reviews)` where `month` is the months in which products are most frequently reviewed (month values are integers: ex. 11 = November) and `num_reviews` is the number of reviews for that `month` during the years specified.

In [ ]:
%%spark

#TODO: Create home_kitch_three_sdf


In [ ]:
%spark -o home_kitch_three_sdf

In [ ]:

# 5 points
grader.grade(test_case_id = 'test_home_kitch_three', answer = home_kitch_three_sdf)

## Step 3: The Riveting Reviews [30 pts]


### Step 3.1: The Short Summary [5 pts]

You don't always have time to read an entire review. When you're about to buy a product you love, you may scroll through some reviews, eyeballing the summaries and how helpful the reviews are. Eyeballing a few reviews that comment on the usability of the product like "cheap but works" or "effective" or "waste of money" can help you make your decision. Your task is to count the number of reviews per category where the summary length is less than 10 characters and it is considered useful.

```
+-------------------+--------------------+
|           category|     helpful_reviews|
+-------------------+--------------------|
+-------------------+--------------------+
```


Name this calculated count of reviews as helpful_reviews and rank categories in descending order of helpful_reviews.


Useful rating is defined as a list - [no. of helpful votes, total no,of votes. For example, a [2,3] means 2 out of 3 others found this review helpful. So, to be considered useful a review must have 1 or more total votes, and 1 or more helpful votes.

In [ ]:
%%spark

#TODO: Create summary_sdf


In [ ]:
%spark -o summary_sdf

In [ ]:

# 5 points
grader.grade(test_case_id = 'test_summary', answer = summary_sdf)

### Step 3.2: The Review Ratio [10 pts]

Find how helpful a review is in terms of a ratio, and return the top 10 most helpful reviews, the corresponding reviewerID and the helpfulness ratio. The helpfulness ratio can be calulated as:





 $\frac{Number\ of \ helpful \ votes}{Total \ number \ of \ votes}$


The following columns are required in your top_10_helpful_sdf: reviewerID, reviewText, helpful_ratio. Make sure to sort the dataframe in descending order of helpful_ratio as well as descending order of reviewerID.

**Note:** Ensure to only use useful ratings when computing the ratio

In [ ]:
%%spark
#TODO: Create top_10_helpful_sdf


In [ ]:
%spark -o top10_helpful_sdf

In [ ]:

# 10 points
grader.grade(test_case_id = 'test_helpful_ratio', answer = top10_helpful_sdf)

### Step 3.3: The Sensible Sentiment

We would like to determine the sentiment of some of the reviews across the categories. Sentiment Analysis is used with textual data very regularly. A common application includes determing whether the review associated with a product is positive, negative or neutral. We are going to attempt to determine the sentiment associated with the top 10 most helpful reviews from the food category, based on the ratio calculated in the previous section. 

#### Step 3.3.1: The Top Ten [5 pts]

Retrieve the asin and reviewText for the top 10 most helpful reviews for the food category, where how helpful a review is determined by the ratio calculated in the previous question. Sort in descending order of helpfulness as well as reviewerID. 

In [ ]:
%%spark
#TODO : Create top10_sdf


In [ ]:
%spark -o top10_sdf

In [ ]:

# 5 points
grader.grade(test_case_id = 'test_top10_helpful', answer = top10_sdf)

#### Step 3.3.2: Positively Positive [10 pts]

Recall from your previous assignment, you were asked to return a tokenized list of words (removing stop words and checking for alphabets). 
 
Using the pandas dataframe created from the previous section, create a new column called 'tokenized' which contains the tokens from the reviewText. (Hint: Use the apply function to call the custom tokenize function from HW2). 

Sentiment analysis is a a natural language processing (NLP) technique that attempts to identify the emotional tone behind a body of text. In this case, we will try to determine how posiitve a review is.


We will be calculating the sentiment associated with each of these reviews. To read more about sentiment analysis, [click here](https://towardsdatascience.com/sentiment-analysis-concept-analysis-and-applications-6c94d6f58c17).

In [ ]:
import nltk
from nltk.corpus import stopwords
nltk.download('stopwords')
nltk.download('punkt')
nltk.download('vader_lexicon')
stopwords = set(stopwords.words('english'))

In [ ]:
#Convert to dataframe & tokenize list 
#TODO: Add your custom tokenize content function 
def tokenize_content(content):
  pass

  
#TODO: Apply your function to reviewText




Run the cell below to understand how to the inbuilt Sentiment Analyzer works. As you can see below, the compound variable contains the overall sentiment of the sentence "Wow, NLTK is really powerful!".





In [ ]:
#EXAMPLE CELL
from nltk.sentiment import SentimentIntensityAnalyzer
sia = SentimentIntensityAnalyzer()
sia.polarity_scores("Wow, NLTK is really powerful!")

{'neg': 0.0, 'neu': 0.295, 'pos': 0.705, 'compound': 0.8012}

Your task is to now add a column to your dataframe called sentiment, and calculate the sentiment associated with each review. Sort this dataframe from highest sentiment to lowest.

Now, moving back into SQL, we need you to run a query on the product_reviews_sdf to retrieve the titles of 2 products with the highest sentiment. Use the asins from the first two rows of your sorted pandas dataframes, and retrieve the just product titles. Save this result into top2_sentiment_sdf. Please do not hardcode the values of the titles, we expect an SQL approach to obtain the titles. However, you may use ASIN values that you obtain from the top10_tokenize_sdf in your SQL query.

You will be passing two results to the autograder : top10_tokenize_sdf (asin, reviewText, tokenized, sentiment columns), top2_sentiment_sdf (only title column)

In [ ]:
sia = SentimentIntensityAnalyzer()
def retrieve_sentiment(content):
  pass

In [ ]:
# TODO: Create sentiment column and sort top10_tokenize_sdf by sentiment 



In [ ]:
%%spark
#TODO: Create top2_sentiment_sdf


In [ ]:
%spark -o top2_sentiment_sdf

In [ ]:

# 10 points
grader.grade(test_case_id = 'test_sentiment_positive', answer = (top2_sentiment_sdf,top10_tokenize_sdf))



## Step 4: Candy Land [26 pts]

In this section, we will be working with a dataset from a poll where each row holds a person's favorite and second favorite candy/sweet. 

Let's introduce a fun little concept called the Bacon Number! The Bacon number of an actor or actress is the number of degrees of separation they have from actor Kevin Bacon, as defined by the game known as Six Degrees of Kevin Bacon. For example, Kevin Bacon's Bacon number is 0. If an actor works in a movie with Kevin Bacon, the actor's Bacon number is 1. If an actor works with an actor who worked with Kevin Bacon in a movie, the first actor's Bacon number is 2, and so forth.

How do we implement the "Super Bacon" for our dataset though? We define this number as follows: if candy A is someone's favorite candy and candy B is that person's second favorite candy while candy B is someone else's favorite candy and candy C is their second favorite candy, then the super bacon of C with respect to A will be 2.

Now to calculate this number, we'll use the concepts of graphs and BFS!

In [ ]:
!pip install pyspark

In [ ]:
from pyspark.sql import SparkSession

appName = "PySpark"

# Create Spark session
spark = SparkSession.builder.appName(appName).getOrCreate()

### Step 4.1 “Traversing” a Graph

Let's review how BFS works!

In [ ]:
import pandas as pd
from IPython.display import Image as I

bfsgif =\
'https://upload.wikimedia.org/wikipedia/commons/5/5d/Breadth-First-S'+\
'earch-Algorithm.gif'
dfsgif=\
'https://upload.wikimedia.org/wikipedia/commons/7/7f/Depth-First-Search.gif'


#### Step 4.1.1 Intro to Distributed Breadth-First Search


To start off, we will be implementing a graph traversal algorithm known as Breadth First Search. It works in a way that's equivalent to how a stain spreads on a white t-shirt. Take a look at the graph below:

<p align = "center">
<img src = "https://imgur.com/WU3AUwg.png" width= "600" align ="center"/>

* Consider starting BFS from point A (green). This is considered the starting frontier/singular origin node.
* The first round of BFS would involve finding all the nodes directly reachable from A, namely B-F (blue circles). These blue nodes make up the next frontier at depth 1 away from our starting node A.
* The second round would then be identifying the red nodes which are the neighbors of the blue nodes. Now, the red nodes all belong to a frontier 2 depth away from A. Note that node A is also a neighbor of a blue node. However, since it has already been visited, it does not get added to this frontier.

This process continues until all the nodes in the graph have been visited.
If you would like to learn more about BFS, we highly suggest looking [here](https://www.tutorialspoint.com/data_structures_algorithms/breadth_first_traversal.html).


We will now be implementing **spark_bfs(G, N, d)**, our spark flavor of BFS that takes a graph **G**, a set of origin nodes **N**, and a max depth **d**.

In order to write a successful BFS function, you are going to need to figure out 
1. how to keep track of nodes that we have visited
2. how to properly find all the nodes at the next depth
3. how to avoid cycles and ensure that we do not constantly loop through the same edges (take a look at J-K in the graph)

#### [OPTIONAL/ADDITIONAL] BFS vs. DFS Animation 
Run the code cells below to understand the difference between depth and breadth first search! (Source: Wikimedia Commons)

In [ ]:
# Do not edit
print('BFS:')
I(url=bfsgif)

In [ ]:
#Do not edit
print('DFS:')
I(url=dfsgif)

#### Step 4.1.2 Implement One Traversal [6 pts]

To break down this process, let's think about how we would implement a single traversal of the graph. That is given the green node in the graph above, how are we going to get the blue nodes?


Consider the simple graph below **which is different from the graph in the image above**:

In [ ]:
# Do not edit
spark.conf.set("spark.sql.execution.arrow.enabled", "true")

simple = [('A', 'B'),
         ('A', 'C'),
         ('A', 'D'),
         ('C', 'F'),
         ('F', 'A'),
         ('B', 'G'),
         ('G', 'H'),
         ('D', 'E')]

simple_dict = {'from_node': ['A', 'A', 'A', 'C', 'F', 'B', 'G', 'D'],
       'to_node': ['B', 'C', 'D', 'F', 'A', 'G', 'H', 'E']}

simple_graph_df = pd.DataFrame.from_dict(simple_dict)
simple_graph_sdf = spark.createDataFrame(simple_graph_df)
simple_graph_sdf.show()

+---------+-------+
|from_node|to_node|
+---------+-------+
|        A|      B|
|        A|      C|
|        A|      D|
|        C|      F|
|        F|      A|
|        B|      G|
|        G|      H|
|        D|      E|
+---------+-------+



As you can see, each row of this dataframe represents an edge between two nodes. Here, we are looking at a directed graph, which means that A-->B does not represent the same edge as B-->A.
Let's define our starting node as follows:

In [ ]:
smallOrig = [{'node': 'A'}]

Then, bfs with graph G, starting from smallOrig to depth 1, or spark_bfs(G, smallOrig, 1) would output as follows:

In [ ]:
# Do not edit
simple_1_round_dict = {'node': ['B', 'D', 'C', 'A'],
       'distance': [1, 1, 1, 0]}
simple_1_round_bfs_df = pd.DataFrame.from_dict(simple_1_round_dict)
simple_1_round_bfs_sdf = spark.createDataFrame(simple_1_round_bfs_df)
simple_1_round_bfs_sdf.show()

+----+--------+
|node|distance|
+----+--------+
|   B|       1|
|   D|       1|
|   C|       1|
|   A|       0|
+----+--------+



As you can see, this dataframe logs each node with its corresponding distance away from A. Moreover, we also know that these nodes are **visited**. 

Hopefully, you can see how we can use our original graph and this new information to find the nodes at depth two. 

This is exactly what we will try to accomplish with **spark_bfs_1_round(visited_nodes)** which will ultimately be the inner function of **spark_bfs** that we use to perform exactly one traversal of a graph.

**TODO**: Write **spark_bfs_1_round(visted_nodes)** that takes the currently dataframe of visited_nodes, performs one round of BFS, and returns an updated visited nodes dataframe. You should assume that a temporary sdf `G` already exists.

In [ ]:
def spark_bfs_1_round(visited_nodes):
  """
  :param visited_nodes: dataframe with columns node and distance
  :return: dataframe of updated visuted nodes, with columns node and distance
  """
  # TODO: Complete this function to implement 1 round of BFS


Now, run the inner function on simple_1_round_bfs_sdf (i.e. result of 1 round of BFS on the simple graph) and store the results in simple_bfs_result. This is ultimately what the output of BFS to depth 2 should look like.

In [ ]:
# TODO: Run spark_bfs_1_round on simple_1_round_bfs_sdf


Convert this result to Pandas, sort your dataframe by ***node*** descending, and submit it to the autograder.

In [ ]:
# TODO: Convert simple_bfs_result to Pandas sorted by node
simple_bfs_test = ...

In [ ]:
# 6 points
grader.grade(test_case_id = 'checksimpleBFS', answer = simple_bfs_test)

#### Step 4.1.3 Full BFS Implementation

Now, we will fully implement **spark_bfs**. This function should iteratively call your implemented version of **spark_bfs_1_round** and ultimately return the output of this function at **max_depth**.

You are also responsible for initializing the starting dataframe, that is converting the list of origin nodes into a spark dataframe with the nodes logged at distance 0.

Consider the following: 

```
schema = StructType([
            StructField("node", StringType(), True)
        ])

    my_sdf = spark.createDataFrame(origins, schema)
```

The schema ultimately specifies the structure of the Spark DataFrame with a string `node` column. It then calls **spark.createDataFrame** to map this schema to the **origins** nodes. Also, you are responsible for ensuring that a view of your graph is available within this function. (Note: you will also need to add in a distance column)

**TODO:** implement **spark_bfs(G,origins,max_depth)** and run on **candy_graph_sdf** initalized in 4.3. Note: you may want to run tests on the **simple_graph** example as the `candy_graph_sdf` will take quite some time to run.

These imports might be useful: 
`from pyspark.sql.types import StructType, StructField, StringType, IntegerType`

In [ ]:
from pyspark.sql.types import StructType, StructField, StringType, IntegerType
# TODO: iterative search over undirected graph
# Worth 5 points directly, but will be needed later

def spark_bfs(G, origins, max_depth):
  """ runs distributed BFS to a specified max depth

  :param G: graph dataframe from 4.3
  :param origins: list of origin nodes stored as {"node": nodeValue}
  :param max_depth: integer value of max depth to run BFS to
  :return: dataframe with columns node, distance of all visited nodes
  """

Test that this function works on the simple example.

In [ ]:
simple_bfs_iterative_result = spark_bfs(simple_graph_sdf, smallOrig, 3)
simple_bfs_iterative_result.show()

### Step 4.2: Popular Candies [5 pts]

Download the `candy_graph_extra.csv` dataset from [here](https://drive.google.com/file/d/1ZjpdZ9wPIMAXvHsr8aocg4IgYtkcLywp/view?usp=sharing), and upload it to the content folder of this Colab notebook.

It's time to take a look at our candy dataset. Let's load the data to **candy_sdf** and see how it looks.

In [ ]:
candy_sdf = spark.read.csv("/content/candy_graph_extra.csv", header=True)

In [ ]:
candy_sdf.show()

Let's take a look at which candies were the most popular. 

***TODO***: Count the number of times that each candy was chosen as their favorite (each candy appears in the FirstCandy column). Return a dataframe with schema (FirstCandy,Votes) where Votes is the number of times that each candy was chosen as a person's favorite. Order your dataframe by Votes.

In [ ]:
# TODO


Convert the dataframe to pandas and pass it to the grader.

In [ ]:
count_df = ...

In [ ]:
grader.grade(test_case_id = 'checkCandyCount', answer = count_df)

Now that we have the Candy dataset, let's convert it to a graph sdf just like the one we had in section 4.1 (P.S. it's not as hard as it sounds).

**TODO:** Create **candy_graph_sdf** that has the columns *from_node* and *to_node*. from_node has all the entries from the *FirstCandy* column and to_node has all the entries from the *SecondCandy* column.

In [ ]:
# TODO

In [ ]:
grader.grade(test_case_id = 'checkCandyGraph', answer = candy_graph_sdf.toPandas())

### Step 4.3: The Super Bacon Search [15 pts]

In the previous section, we found out that the Starburst Sour Chews and Kit Kat Chunky Mug Egg  have been the most popular candies for this group of customers. So let's find out the Super Bacons of candies with respect to Starburst Sour Chews.

First, we'll create an origin node using Starburst Sour Chews.

In [ ]:
orig = [{'node': 'Starburst Sour Chews'}]

Now, run the **spark_bfs()** function on **teams_graph_sdf** using this origin node and a max depth of 5. Store the result in **bfs_5**.

In [ ]:
bfs_5 = spark_bfs(candy_graph_sdf, orig, 5)


In [ ]:
bfs_5.show()

Convert the result to pandas, store it to answer_df and pass this to the grader.

In [ ]:
answer_df = bfs_5.toPandas()

In [ ]:
grader.grade(test_case_id = 'checkBFS', answer = answer_df)

Voila! We're all done.

# HW Submission

Before you submit on Gradescope (you must submit your notebook to receive credit):


1.   Restart and Run-All to make sure there's nothing wrong with your notebook
2.   **Double check that you have the correct PennID (all numbers) in the autograder**. 
3. Make sure you've run all the PennGrader cells
4. Go to the "File" tab at the top left, and click "Download .ipynb" and then "Download .py".  **Rename** the files to "homework3.ipynb" and "homework3.py" respectively and upload them to Gradescope 

**Let the course staff know ASAP if you have any issues submitting, but otherwise best of luck!**